In [1]:
from __future__ import print_function, division
import numpy as np 
import pandas as pd 

import os
        
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

import optuna
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import xgboost as xgb
torch.manual_seed(15)

# Import data

In [2]:
df_classes = pd.read_csv("elliptic_bitcoin_dataset/elliptic_txs_classes.csv")
df_edges = pd.read_csv("elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv")
df_features = pd.read_csv("elliptic_bitcoin_dataset/elliptic_txs_features.csv", header=None)
df_features.columns = ['txId', 'time_step'] + [f'trans_feat_{i}' for i in range(93)] + [f'agg_feat_{i}' for i in range(72)]
df_features = pd.merge(df_features,df_classes,left_on="txId",right_on="txId",how='left')
df_features['class'] = df_features['class'].apply(lambda x: '0' if x == "unknown" else x)

# Train and Test data

In [3]:
tx_features = ["trans_feat_"+str(i) for i in range(93)]
agg_features = ["agg_feat_"+str(i) for i in range(72)]
data = df_features[(df_features['class']=='1') | (df_features['class']=='2')]
X = data[tx_features + agg_features] 
y = data['class']
y = y.apply(lambda x: 0 if x == '2' else 1 )
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=15,shuffle=False)

# GCN

In [4]:
classes = df_classes
edgelist = df_edges
features = df_features
start_ts = 0
end_ts = 34

In [5]:

num_features = features.shape[1]
num_tx = features.shape[0]	
total_tx = list(classes.index)

# select only the transactions which are labelled
labelled_classes = classes[classes['class'] != 0]
labelled_tx = list(labelled_classes.index)

# to calculate a list of adjacency matrices for the different timesteps

adj_mats = []
features_labelled_ts = []
classes_ts = []
num_ts = 49 # number of timestamps from the paper

for ts in range(start_ts, end_ts):
    features_ts = features[features['time_step'] == ts+1]
    tx_ts = list(features_ts.index)

    labelled_tx_ts = [tx for tx in tx_ts if tx in set(labelled_tx)]

    # adjacency matrix for all the transactions
    # we will only fill in the transactions of this timestep which have labels and can be used for training
    adj_mat = pd.DataFrame(np.zeros((num_tx, num_tx)), index = total_tx, columns = total_tx)

    edgelist_labelled_ts = edgelist.loc[edgelist.index.intersection(labelled_tx_ts).unique()]
    for i in range(edgelist_labelled_ts.shape[0]):
        adj_mat.loc[edgelist_labelled_ts.index[i], edgelist_labelled_ts.iloc[i]['txId2']] = 1

    adj_mat_ts = adj_mat.loc[labelled_tx_ts, labelled_tx_ts]
    features_l_ts = features.loc[labelled_tx_ts]

    adj_mats.append(adj_mat_ts)
    features_labelled_ts.append(features_l_ts)
    classes_ts.append(classes.loc[labelled_tx_ts])


/Users/yco/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


: 

In [ ]:
dataSet = load_data(df_classes, df_edges,df_features, 0,34)

KeyError: 1